# NBA Player Data Analysis Project

## Project Outline

1. **Introduction**
   - Overview of the project
   - Goals and objectives

2. **Data Collection**
   - Import necessary libraries
   - Fetch player data from the NBA API

3. **Data Processing**
   - Load data into a Pandas DataFrame
   - Data cleaning and transformation

4. **Data Analysis**
   - Exploratory data analysis (EDA)
   - Visualizations

5. **Conclusion**
   - Summary of findings
   - Future work


## 0. Environment

### Python

This project is written in Python, which means that Python must be installed in your environment to run the project. The minimum supported version is 3.10.

#### Windows

You can use the Windows package manager `winget`, or the [installer](https://www.python.org/downloads/windows/) from the website.
```powershell
# you can change the version in the package name to your desired version
winget install Python.Python.3.12
```

#### MacOS
Python is already installed by default on recent versions of MacOS. If you have an older version that is not supported, you can use the [Homebrew](https://brew.sh/) package manager to install it, or the [installer](https://www.python.org/downloads/macos/) from the website.
```zsh
brew install python
```

#### Linux
Python is already installed by default on most distributions of Linux. If it isn't, you can use your distribution's package manager to install Python.

### Virtual Environment

It's generally recommended that you use a virtual environment (or venv) for this project. That way, all dependencies can be installed for the project without affecting the rest of your system. You can create a venv with Python:

```bash
python -m venv .venv
```

To activate the virtual environment in your shell, you can use the following commands.

On Windows:

```powershell
.venv\Scripts\activate
```

On other operating systems:

```bash
.venv/bin/activate
```

### Dependencies

This project uses [Poetry](https://python-poetry.org/) to manage its dependencies. You can install the dependencies with the `poetry` command:

`poetry install`

If you don't want to use Poetry, a `requirements.txt` is also provided. You can install this using `pip`:

`pip install -r requirements.txt`

### Imports

In [ ]:
import json
import time
import os
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
from typing import Dict

### Environment Variables

We will load all our environment variables from a `.env` file, if one is provided.

If database information is provided, all dataframes used for analysis are uploaded to it. We use [Microsoft SQL Server](https://www.microsoft.com/en-us/sql-server/sql-server-downloads) by default but any kind of database is supported.

In [ ]:
from dotenv import load_dotenv
load_dotenv()
DB_TYPE = os.getenv("DB_TYPE", "sqlserver")
DB_USER = os.getenv("DB_USER", "sqladmin")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "1433")
DB_NAME = os.getenv("DB_NAME", "dataframes")
DB_DRIVER = os.getenv("DB_DRIVER") # some databases require a database driver

### Presentation

By default, Pandas dataframes are truncated when they are printed. We want to be able to view all of the data at once, so we embed the dataframe in a scrollable element.

In [ ]:
from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell

def custom_scrollable_display(df: pd.DataFrame, max_height=400):
    """
    Custom display function to render DataFrames as scrollable elements.
    
    Parameters:
    - df: The DataFrame to display.
    - max_height: The maximum height of the scrollable area in pixels.
    """
    style = f"""
    <style>
    .scrollable-dataframe {{
        display: inline-block;
        white-space: nowrap;
        overflow-x: scroll;
        max-height: {max_height}px;
        overflow-y: scroll;
    }}
    </style>
    """
    display(HTML(style + f'<div class="scrollable-dataframe">{df.to_html()}</div>'))

def custom_display_hook(df):
    custom_scrollable_display(df)
    return ""

# hook up the custom display function to the automatic printer
InteractiveShell.instance().display_formatter.formatters['text/html'].for_type(pd.DataFrame, custom_display_hook);


### Pre-Commit Hooks (Developer Only)

This notebook uses `nbstripout` to strip notebook output from Git commits. If you are committing code, please run the following command to set up the Git filter.

Poetry is required for the pre-commit hooks, so make sure it is installed before you commit code. You will also need to add the plugin `poetry-plugin-export` in order to run the export hook.
```bash
poetry self add poetry-plugin-export
```

In [ ]:
!nbstripout --install
!pre-commit install

## 1. Data Collection

### Fetch Player Data from NBA API

`nba_api` provides static player and team information, which we will download here so that we can reuse it without requesting the API unnecessarily.

In [ ]:
from nba_api.stats.static import players, teams
PLAYERS_LIST_FILE = "../data/players_list.csv"
TEAMS_LIST_FILE = "../data/teams_list.csv"

if os.path.exists(PLAYERS_LIST_FILE):
    players_list = pd.read_csv(PLAYERS_LIST_FILE)
else:
    players_list = pd.DataFrame(players.get_players())
    players_list.to_csv(PLAYERS_LIST_FILE)

if os.path.exists(TEAMS_LIST_FILE):
    teams_list = pd.read_csv(TEAMS_LIST_FILE)
else:
    teams_list = pd.DataFrame(teams.get_teams())
    teams_list.to_csv(TEAMS_LIST_FILE)


### Fetch Game Data



We're only interested in games that are either in the regular season or in the playoffs. We'll add an enum to distinguish the type of game and use it to differentiate them.

In [ ]:
from enum import Enum
class SeasonType(Enum):
    PRESEASON = 1
    REGULAR_SEASON = 2
    ALL_STAR = 3
    PLAYOFFS = 4
    PLAY_IN = 5
    NBA_CUP = 6
class Season():
    def __init__(self, season_id: int) -> None:
        season_id_str = str(season_id)
        self.season_type = SeasonType(int(season_id_str[0]))
        self.season_year = int(season_id_str[1:])

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import LeagueIDNullable, LeagueID
START_SEASON = 2015
END_SEASON = 2023
GAMES_LIST_FILE = "../data/games_list.csv"
if os.path.exists(GAMES_LIST_FILE):
    games_list: pd.DataFrame = pd.read_csv(GAMES_LIST_FILE)
else:
    games_list = pd.DataFrame()
    for season in range(START_SEASON, END_SEASON + 1):
        # put season into the correct form e.g. 2023 -> 2023-24
        season_str = f"{season}-{str(season + 1)[2:]}"
        print(f"Fetching games for season: {season_str}", end="\r")
        gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=season_str, league_id_nullable=LeagueIDNullable.nba)
        games = gamefinder.get_data_frames()[0]
        games_list = pd.concat([games_list, games], ignore_index=True)
        time.sleep(0.6)
    games_list.to_csv(GAMES_LIST_FILE, index=False)
# games_list["SEASON_ID"].unique()
# games_list.loc[(games_list["TEAM_NAME"] == "San Antonio Spurs") & (games_list["SEASON_ID"] == 22023)]
games_list.head()

### Fetch Play by Plays

In [ ]:
from nba_api.stats.endpoints import playbyplayv3
from requests.exceptions import ReadTimeout
# took 483 minutes to download up to 2012
PBP_LIST_FILE = "../data/pbp_list.csv"
if os.path.exists(PBP_LIST_FILE):
    pbp_list = pd.read_csv(PBP_LIST_FILE)
else:
    unique_games_list = games_list.drop_duplicates(subset="GAME_ID")
    pbp_list = pd.DataFrame()
    for index, row in unique_games_list.iterrows():
        err = False
        game_id = row["GAME_ID"]
        game_date = row["GAME_DATE"]
        season_id = row["SEASON_ID"]
        season = Season(season_id)
        if season.season_type != SeasonType.REGULAR_SEASON and season.season_type != SeasonType.PLAYOFFS:
            continue
        print(f"Fetching play by play for game {game_id} on {game_date}", end="\r")
        while True:
            try:
                pbpfinder = playbyplayv3.PlayByPlayV3(f"{game_id:010}")
                break
            except ReadTimeout as e:
                print(f"{e}! Try again")
            except Exception:
                with open("../data/err.log", "a") as f:
                    print(f"{game_id} does not have a play by play", file=f)
                err = True
                break
        if err:
            continue
        pbp = pbpfinder.get_data_frames()[0]
        pbp_list = pd.concat([pbp_list, pbp], ignore_index=True)
        time.sleep(0.6)
    pbp_list.to_csv(PBP_LIST_FILE, index=False)
pbp_list.head()


### Fetch Box Scores

#### Player Track

In [ ]:
from nba_api.stats.endpoints import boxscoreplayertrackv3
from requests.exceptions import ReadTimeout
# took 483 minutes to download up to 2012
BOXSCORE_PT_LIST_FILE = "../data/boxscore_pt_list.csv"
if os.path.exists(BOXSCORE_PT_LIST_FILE):
    boxscore_pt_list = pd.read_csv(BOXSCORE_PT_LIST_FILE)
else:
    unique_games_list = games_list.drop_duplicates(subset="GAME_ID")
    boxscore_pt_list = pd.DataFrame()
    for index, row in unique_games_list.iterrows():
        err = False
        game_id = row["GAME_ID"]
        game_date = row["GAME_DATE"]
        season_id = row["SEASON_ID"]
        season = Season(season_id)
        if season.season_type != SeasonType.REGULAR_SEASON and season.season_type != SeasonType.PLAYOFFS:
            continue
        print(f"Fetching box score for game {game_id} on {game_date}", end="\r")
        while True:
            try:
                boxscorefinder = boxscoreplayertrackv3.BoxScorePlayerTrackV3(f"{game_id:010}")
                break
            except ReadTimeout as e:
                print(f"{e}! Try again")
            except Exception:
                with open("../data/err.log", "a") as f:
                    print(f"{game_id} does not have a player track box score", file=f)
                err = True
                break
        if err:
            continue
        boxscore_pt = boxscorefinder.get_data_frames()[0]
        boxscore_pt_list = pd.concat([boxscore_pt_list, boxscore_pt], ignore_index=True)
        time.sleep(0.6)
    boxscore_pt_list.to_csv(BOXSCORE_PT_LIST_FILE, index=False)
boxscore_pt_list.head()


#### Defensive

In [ ]:
%%script true

from nba_api.stats.endpoints import boxscoredefensivev2
from requests.exceptions import ReadTimeout
# took 483 minutes to download up to 2012
BOXSCORE_DF_LIST_FILE = "../data/boxscore_df_list.csv"
if os.path.exists(BOXSCORE_DF_LIST_FILE):
    boxscore_df_list = pd.read_csv(BOXSCORE_DF_LIST_FILE)
else:
    unique_games_list = games_list.drop_duplicates(subset="GAME_ID")
    boxscore_df_list = pd.DataFrame()
    for index, row in unique_games_list.iterrows():
        err = False
        game_id = row["GAME_ID"]
        game_date = row["GAME_DATE"]
        season_id = row["SEASON_ID"]
        season = Season(season_id)
        if season.season_type != SeasonType.REGULAR_SEASON and season.season_type != SeasonType.PLAYOFFS:
            continue
        print(f"Fetching box score for game {game_id} on {game_date}", end="\r")
        while True:
            try:
                boxscorefinder = boxscoredefensivev2.BoxScoreDefensiveV2(f"{game_id:010}")
                break
            except ReadTimeout as e:
                print(f"{e}! Try again")
            except Exception:
                with open("../data/err.log", "a") as f:
                    print(f"{game_id} does not have a defensive box score", file=f)
                err = True
                break
        if err:
            continue
        boxscore_df = boxscorefinder.get_data_frames()[0]
        boxscore_df_list = pd.concat([boxscore_df_list, boxscore_df], ignore_index=True)
        time.sleep(0.6)
    boxscore_df_list.to_csv(BOXSCORE_DF_LIST_FILE, index=False)
boxscore_df_list.head()

#### Traditional

In [ ]:
%%script true

from nba_api.stats.endpoints import boxscoretraditionalv3
from requests.exceptions import ReadTimeout
# took 483 minutes to download up to 2012
BOXSCORE_TD_LIST_FILE = "../data/boxscore_td_list.csv"
if os.path.exists(BOXSCORE_TD_LIST_FILE):
    boxscore_td_list = pd.read_csv(BOXSCORE_TD_LIST_FILE)
else:
    unique_games_list = games_list.drop_duplicates(subset="GAME_ID")
    boxscore_td_list = pd.DataFrame()
    for index, row in unique_games_list.iterrows():
        err = False
        game_id = row["GAME_ID"]
        game_date = row["GAME_DATE"]
        season_id = row["SEASON_ID"]
        season = Season(season_id)
        if season.season_type != SeasonType.REGULAR_SEASON and season.season_type != SeasonType.PLAYOFFS:
            continue
        print(f"Fetching box score for game {game_id} on {game_date}", end="\r")
        while True:
            try:
                boxscorefinder = boxscoretraditionalv3.BoxScoreTraditionalV3(f"{game_id:010}")
                break
            except ReadTimeout as e:
                print(f"{e}! Try again")
            except Exception:
                with open("../data/err.log", "a") as f:
                    print(f"{game_id} does not have a traditional box score", file=f)
                err = True
                break
        if err:
            continue
        boxscore_td = boxscorefinder.get_data_frames()[0]
        boxscore_td_list = pd.concat([boxscore_td_list, boxscore_td], ignore_index=True)
        time.sleep(0.6)
    boxscore_td_list.to_csv(BOXSCORE_TD_LIST_FILE, index=False)
boxscore_td_list.head()

#### Advanced

In [ ]:
%%script true

from nba_api.stats.endpoints import boxscoreadvancedv3
from requests.exceptions import ReadTimeout
# took 483 minutes to download up to 2012
BOXSCORE_AD_LIST_FILE = "../data/boxscore_ad_list.csv"
if os.path.exists(BOXSCORE_AD_LIST_FILE):
    boxscore_ad_list = pd.read_csv(BOXSCORE_AD_LIST_FILE)
else:
    unique_games_list = games_list.drop_duplicates(subset="GAME_ID")
    boxscore_ad_list = pd.DataFrame()
    for index, row in unique_games_list.iterrows():
        err = False
        game_id = row["GAME_ID"]
        game_date = row["GAME_DATE"]
        season_id = row["SEASON_ID"]
        season = Season(season_id)
        if season.season_type != SeasonType.REGULAR_SEASON and season.season_type != SeasonType.PLAYOFFS:
            continue
        print(f"Fetching box score for game {game_id} on {game_date}", end="\r")
        while True:
            try:
                boxscorefinder = boxscoreadvancedv3.BoxScoreAdvancedV3(f"{game_id:010}")
                break
            except ReadTimeout as e:
                print(f"{e}! Try again")
            except Exception:
                with open("../data/err.log", "a") as f:
                    print(f"{game_id} does not have an advanced box score", file=f)
                err = True
                break
        if err:
            continue
        boxscore_ad = boxscorefinder.get_data_frames()[0]
        boxscore_ad_list = pd.concat([boxscore_ad_list, boxscore_ad], ignore_index=True)
        time.sleep(0.6)
    boxscore_ad_list.to_csv(BOXSCORE_AD_LIST_FILE, index=False)
boxscore_ad_list.head()

#### Four Factors

In [ ]:
%%script true

from nba_api.stats.endpoints import boxscorefourfactorsv3
from requests.exceptions import ReadTimeout
# took 483 minutes to download up to 2012
BOXSCORE_FF_LIST_FILE = "../data/boxscore_ff_list.csv"
if os.path.exists(BOXSCORE_FF_LIST_FILE):
    boxscore_ff_list = pd.read_csv(BOXSCORE_FF_LIST_FILE)
else:
    unique_games_list = games_list.drop_duplicates(subset="GAME_ID")
    boxscore_ff_list = pd.DataFrame()
    for index, row in unique_games_list.iterrows():
        err = False
        game_id = row["GAME_ID"]
        game_date = row["GAME_DATE"]
        season_id = row["SEASON_ID"]
        season = Season(season_id)
        if season.season_type != SeasonType.REGULAR_SEASON and season.season_type != SeasonType.PLAYOFFS:
            continue
        print(f"Fetching box score for game {game_id} on {game_date}", end="\r")
        while True:
            try:
                boxscorefinder = boxscorefourfactorsv3.BoxScoreFourFactorsV3(f"{game_id:010}")
                break
            except ReadTimeout as e:
                print(f"{e}! Try again")
            except Exception:
                with open("../data/err.log", "a") as f:
                    print(f"{game_id} does not have a four factors box score", file=f)
                err = True
                break
        if err:
            continue
        boxscore_ff = boxscorefinder.get_data_frames()[0]
        boxscore_ff_list = pd.concat([boxscore_ff_list, boxscore_ff], ignore_index=True)
        time.sleep(0.6)
    boxscore_ff_list.to_csv(BOXSCORE_FF_LIST_FILE, index=False)
boxscore_ff_list.head()

#### Hustle

In [ ]:
%%script true

from nba_api.stats.endpoints import boxscorehustlev2
from requests.exceptions import ReadTimeout
# took 483 minutes to download up to 2012
BOXSCORE_HS_LIST_FILE = "../data/boxscore_hs_list.csv"
if os.path.exists(BOXSCORE_HS_LIST_FILE):
    boxscore_hs_list = pd.read_csv(BOXSCORE_HS_LIST_FILE)
else:
    unique_games_list = games_list.drop_duplicates(subset="GAME_ID")
    boxscore_hs_list = pd.DataFrame()
    for index, row in unique_games_list.iterrows():
        err = False
        game_id = row["GAME_ID"]
        game_date = row["GAME_DATE"]
        season_id = row["SEASON_ID"]
        season = Season(season_id)
        if season.season_type != SeasonType.REGULAR_SEASON and season.season_type != SeasonType.PLAYOFFS:
            continue
        print(f"Fetching box score for game {game_id} on {game_date}", end="\r")
        while True:
            try:
                boxscorefinder = boxscorehustlev2.BoxScoreHustleV2(f"{game_id:010}")
                break
            except ReadTimeout as e:
                print(f"{e}! Try again")
            except Exception:
                with open("../data/err.log", "a") as f:
                    print(f"{game_id} does not have a hustle box score", file=f)
                err = True
                break
        if err:
            continue
        boxscore_hs = boxscorefinder.get_data_frames()[0]
        boxscore_hs_list = pd.concat([boxscore_hs_list, boxscore_hs], ignore_index=True)
        time.sleep(0.6)
    boxscore_hs_list.to_csv(BOXSCORE_HS_LIST_FILE, index=False)
boxscore_hs_list.head()

#### Matchups

In [ ]:
%%script true

from nba_api.stats.endpoints import boxscorematchupsv3
from requests.exceptions import ReadTimeout
# took 483 minutes to download up to 2012
BOXSCORE_MU_LIST_FILE = "../data/boxscore_mu_list.csv"
if os.path.exists(BOXSCORE_MU_LIST_FILE):
    boxscore_mu_list = pd.read_csv(BOXSCORE_MU_LIST_FILE)
else:
    unique_games_list = games_list.drop_duplicates(subset="GAME_ID")
    boxscore_mu_list = pd.DataFrame()
    for index, row in unique_games_list.iterrows():
        err = False
        game_id = row["GAME_ID"]
        game_date = row["GAME_DATE"]
        season_id = row["SEASON_ID"]
        season = Season(season_id)
        if season.season_type != SeasonType.REGULAR_SEASON and season.season_type != SeasonType.PLAYOFFS:
            continue
        print(f"Fetching box score for game {game_id} on {game_date}", end="\r")
        while True:
            try:
                boxscorefinder = boxscorematchupsv3.BoxScoreMatchupsV3(f"{game_id:010}")
                break
            except ReadTimeout as e:
                print(f"{e}! Try again")
            except Exception:
                with open("../data/err.log", "a") as f:
                    print(f"{game_id} does not have a matchup box score", file=f)
                err = True
                break
        if err:
            continue
        boxscore_mu = boxscorefinder.get_data_frames()[0]
        boxscore_mu_list = pd.concat([boxscore_mu_list, boxscore_mu], ignore_index=True)
        time.sleep(0.6)
    boxscore_mu_list.to_csv(BOXSCORE_MU_LIST_FILE, index=False)
boxscore_mu_list.head()

#### Miscellaneous

In [ ]:
%%script true

from nba_api.stats.endpoints import boxscoremiscv3
from requests.exceptions import ReadTimeout
# took 483 minutes to download up to 2012
BOXSCORE_MS_LIST_FILE = "../data/boxscore_ms_list.csv"
if os.path.exists(BOXSCORE_MS_LIST_FILE):
    boxscore_ms_list = pd.read_csv(BOXSCORE_MS_LIST_FILE)
else:
    unique_games_list = games_list.drop_duplicates(subset="GAME_ID")
    boxscore_ms_list = pd.DataFrame()
    for index, row in unique_games_list.iterrows():
        err = False
        game_id = row["GAME_ID"]
        game_date = row["GAME_DATE"]
        season_id = row["SEASON_ID"]
        season = Season(season_id)
        if season.season_type != SeasonType.REGULAR_SEASON and season.season_type != SeasonType.PLAYOFFS:
            continue
        print(f"Fetching box score for game {game_id} on {game_date}", end="\r")
        while True:
            try:
                boxscorefinder = boxscoremiscv3.BoxScoreMiscV3(f"{game_id:010}")
                break
            except ReadTimeout as e:
                print(f"{e}! Try again")
            except Exception:
                with open("../data/err.log", "a") as f:
                    print(f"{game_id} does not have a miscellaneous box score", file=f)
                err = True
                break
        if err:
            continue
        boxscore_ms = boxscorefinder.get_data_frames()[0]
        boxscore_ms_list = pd.concat([boxscore_ms_list, boxscore_ms], ignore_index=True)
        time.sleep(0.6)
    boxscore_ms_list.to_csv(BOXSCORE_MS_LIST_FILE, index=False)
boxscore_ms_list.head()

#### Scoring

In [ ]:
%%script true

from nba_api.stats.endpoints import boxscorescoringv3
from requests.exceptions import ReadTimeout
# took 483 minutes to download up to 2012
BOXSCORE_SC_LIST_FILE = "../data/boxscore_sc_list.csv"
if os.path.exists(BOXSCORE_SC_LIST_FILE):
    boxscore_sc_list = pd.read_csv(BOXSCORE_SC_LIST_FILE)
else:
    unique_games_list = games_list.drop_duplicates(subset="GAME_ID")
    boxscore_sc_list = pd.DataFrame()
    for index, row in unique_games_list.iterrows():
        err = False
        game_id = row["GAME_ID"]
        game_date = row["GAME_DATE"]
        season_id = row["SEASON_ID"]
        season = Season(season_id)
        if season.season_type != SeasonType.REGULAR_SEASON and season.season_type != SeasonType.PLAYOFFS:
            continue
        print(f"Fetching box score for game {game_id} on {game_date}", end="\r")
        while True:
            try:
                boxscorefinder = boxscorescoringv3.BoxScoreScoringV3(f"{game_id:010}")
                break
            except ReadTimeout as e:
                print(f"{e}! Try again")
            except Exception:
                with open("../data/err.log", "a") as f:
                    print(f"{game_id} does not have a scoring box score", file=f)
                err = True
                break
        if err:
            continue
        boxscore_sc = boxscorefinder.get_data_frames()[0]
        boxscore_sc_list = pd.concat([boxscore_sc_list, boxscore_sc], ignore_index=True)
        time.sleep(0.6)
    boxscore_sc_list.to_csv(BOXSCORE_SC_LIST_FILE, index=False)
boxscore_sc_list.head()

#### Usage

In [ ]:
%%script true

from nba_api.stats.endpoints import boxscoreusagev3
from requests.exceptions import ReadTimeout
# took 483 minutes to download up to 2012
BOXSCORE_US_LIST_FILE = "../data/boxscore_us_list.csv"
if os.path.exists(BOXSCORE_US_LIST_FILE):
    boxscore_us_list = pd.read_csv(BOXSCORE_US_LIST_FILE)
else:
    unique_games_list = games_list.drop_duplicates(subset="GAME_ID")
    boxscore_us_list = pd.DataFrame()
    for index, row in unique_games_list.iterrows():
        err = False
        game_id = row["GAME_ID"]
        game_date = row["GAME_DATE"]
        season_id = row["SEASON_ID"]
        season = Season(season_id)
        if season.season_type != SeasonType.REGULAR_SEASON and season.season_type != SeasonType.PLAYOFFS:
            continue
        print(f"Fetching box score for game {game_id} on {game_date}", end="\r")
        while True:
            try:
                boxscorefinder = boxscoreusagev3.BoxScoreUsageV3(f"{game_id:010}")
                break
            except ReadTimeout as e:
                print(f"{e}! Try again")
            except Exception:
                with open("../data/err.log", "a") as f:
                    print(f"{game_id} does not have a usage box score", file=f)
                err = True
                break
        if err:
            continue
        boxscore_us = boxscorefinder.get_data_frames()[0]
        boxscore_us_list = pd.concat([boxscore_us_list, boxscore_us], ignore_index=True)
        time.sleep(0.6)
    boxscore_us_list.to_csv(BOXSCORE_US_LIST_FILE, index=False)
boxscore_us_list.head()

## 2. Data Processing

### Recency

Our analysis will consider only the last two years, so we'll get rid of data from before that.

In [ ]:
season_year = games_list['SEASON_ID'].astype(str).str[1:].astype(int)
games_list['season_year'] = season_year
current_season_year = 2023  # Replace with the current season's start year
games_list = games_list[games_list['season_year'].isin([current_season_year, current_season_year - 1])]

In [ ]:
pbp_list = pbp_list[pbp_list['gameId'].isin(games_list["GAME_ID"].unique())]

In [ ]:
boxscore_pt_list = boxscore_pt_list[boxscore_pt_list['gameId'].isin(games_list["GAME_ID"].unique())]

### Unnecessary Columns

Some of this data isn't useful to us, so we'll drop it to ignore the noise.

In [ ]:
pbp_columns_to_drop = ["actionNumber", "pointsTotal", "videoAvailable", "actionId", "playerNameI", "teamTricode"]
pbp_list.drop(columns=[col for col in pbp_columns_to_drop if col in pbp_list.columns], inplace=True)
pbp_list.head()

In [ ]:
bspt_columns_to_drop = ["teamCity", "teamName", "teamTricode", "teamSlug", "playerNameI", "teamTricode", "playerSlug", "jerseyNum"]
boxscore_pt_list.drop(columns=[col for col in bspt_columns_to_drop if col in boxscore_pt_list.columns], inplace=True)
boxscore_pt_list.head()

### Categorization

To save on memory, we will also turn variables that can be understood as categorical variables into that type.

In [ ]:
pbp_categorical_columns = ["gameId", "teamId", "shotResult", "isFieldGoal", "location", "actionType", "subType", "personId", "playerName"]
pbp_list[pbp_categorical_columns] = pbp_list[pbp_categorical_columns].astype("category")
pbp_list.head()

In [ ]:
bspt_categorical_columns = ["gameId", "teamId", "personId", "firstName", "familyName", "nameI",  "position"]
boxscore_pt_list[bspt_categorical_columns] = boxscore_pt_list[bspt_categorical_columns].astype("category")
boxscore_pt_list.head()

### Clock

We'll transform the clock data from a string into the total number of seconds.

In [ ]:
if pbp_list["clock"].dtype != "int64":
    pbp_list["clock"] = pbp_list["clock"].astype(str)
    pbp_list["minutes"] = pbp_list["clock"].str[2:4].astype(int)
    pbp_list["seconds"] = pbp_list["clock"].str[5:7].astype(int)
    pbp_list["clock"] = pbp_list["minutes"] * 60 + pbp_list["seconds"]
    pbp_list.drop(columns=["minutes", "seconds"], inplace=True)
pbp_list["clock"].head()

In [ ]:
if boxscore_pt_list["minutes"].dtype != "int64":
    boxscore_pt_list["minutes"] = boxscore_pt_list["minutes"].astype(str)
    boxscore_pt_list["mins"] = boxscore_pt_list["minutes"].str[:-3].astype(int)
    boxscore_pt_list["seconds"] = boxscore_pt_list["minutes"].str[-2:].astype(int)
    boxscore_pt_list["minutes"] = boxscore_pt_list["mins"] * 60 + boxscore_pt_list["seconds"]
    boxscore_pt_list.drop(columns=["mins", "seconds"], inplace=True)
boxscore_pt_list["minutes"].head()

## 3. Data Analysis

### Defensive Plus/Minus in the Paint

In order to determine which players are the best in protecting the paint, we will calculate how many field goals are scored against them in the paint when they are on the floor vs. when they are not, creating a plus/minus. Note that this is similar to, but distinct from, the existing defensive rim field goals stat. That stat is specific to shots that the players were identified as defending. This stat is broader and encompasses all points in the paint, which allows us to capture paint impact beyond defending the shooter.

We'll look at a range of players so that we can have a general view of how the stats look on them.
- Rudy Gobert: defensive player of the year
- Walter Kessler: a young elite rim protector
- Evan Mobley: a young versatile defender with a lot of upside
- Ivica Zubac: one of the league's most underrated rim protectors
- Jusuf Nurkic: veteran traditional rim protector who is past his prime

In [ ]:
def defpmpaint(player_id: int, player_name: str) -> pd.DataFrame:
    bs_player_games = boxscore_pt_list[boxscore_pt_list['personId'] == player_id]
    bs_player_games = bs_player_games[bs_player_games["comment"].isna()]

    games_with_player = pbp_list[pbp_list['gameId'].isin(bs_player_games['gameId'].unique())]

    #games_with_player = pbp_list.groupby("gameId", observed=True).filter(lambda row: player_id in row["personId"].values)
    games_with_player["gameId"].unique()
    games_with_player = games_with_player[
        (games_with_player['actionType'].isin(['Substitution', 'Missed Shot', 'Made Shot']))
    ]

    # Further filter for shots by the opposing team and shot distance < 3
    games_with_player = games_with_player[
        ((games_with_player['actionType'].isin(['Missed Shot', 'Made Shot']))) |
        (games_with_player['actionType'] == 'Substitution')
    ]
    if 'totalMinutes' not in games_with_player.columns:
        total_minutes_per_game = games_with_player.groupby('gameId', observed=True)['period'].nunique() * 720
        games_with_player = games_with_player.merge(total_minutes_per_game.rename('totalMinutes'), on='gameId')

    unique_games = games_with_player[['gameId', 'totalMinutes']].drop_duplicates()

    total_minutes_all_games = unique_games['totalMinutes'].sum()
    total_minutes_all_games
    games_with_player["isOnFloor"] = False
    games_with_player["playerTeamId"] = None

    for game_id, game_data in games_with_player.groupby('gameId', observed=True):

        # check if the player is starting and set onFloor to true or false based on that
        game_bs = boxscore_pt_list[boxscore_pt_list["gameId"] == game_id]
        player_bs = game_bs.loc[game_bs["personId"] == player_id]
        player_team_id = player_bs.loc[:, "teamId"].values[0]
        player_position = player_bs.loc[:, "position"]
        games_with_player.loc[games_with_player['gameId'] == game_id, 'playerTeamId'] = player_team_id
        # print(game_id)
        # print(game_bs)
        on_floor = not isinstance(player_position.values[0], float)
        #print(player_position.values[0], on_floor)
        # player_team_id = None
        indices = game_data.index.tolist()
        
        for i in range(len(game_data)):
            row = game_data.iloc[i]
            # if player_team_id is None and row["personId"] == player_id:
            #     # Set the player's teamId from this row (assumed to be correct for the whole game)
            #     player_team_id = row['teamId']
            #     games_with_player.loc[games_with_player['gameId'] == game_id, 'playerTeamId'] = player_team_id
            
            if row['actionType'] == 'Substitution':
                sub_description = row['description']
                if f'FOR {player_name}' in sub_description:
                    # print(f"SUB found: {sub_description} in game {game_id}")
                    on_floor = False  # Player subbed out
                elif player_name in sub_description:
                    # print(f"SUB found: {sub_description} in game {game_id}")
                    on_floor = True  # Player subbed in

            # Update the isOnFloor status
            games_with_player.at[indices[i], 'isOnFloor'] = on_floor
    games_with_player.head()
    # filter shots to visitor only
    shots = games_with_player[
        ((games_with_player['actionType'].isin(['Missed Shot', 'Made Shot'])) & 
        (games_with_player['teamId'] != games_with_player["playerTeamId"]))
    ].copy()

    shots.loc[:, 'isLayup'] = shots['subType'].str.contains('Layup', case=False, na=False)
    shots.loc[:, 'isDunk'] = shots['subType'].str.contains('Dunk', case=False, na=False)
    shots.loc[:, 'isHook'] = shots['subType'].str.contains('Hook', case=False, na=False)
    shots.loc[:, 'isJump'] = shots['subType'].str.contains('Jump', case=False, na=False)

    ldh_shots = shots[(shots["isLayup"]) | (shots["isDunk"]) | (shots["isHook"])]

    shots_on_floor = ldh_shots[ldh_shots['isOnFloor']]
    shots_off_floor = ldh_shots[~ldh_shots['isOnFloor']]

    player_minutes = boxscore_pt_list[boxscore_pt_list['personId'] == player_id][['gameId', 'minutes']].set_index('gameId')

    shots_on_floor = shots_on_floor.join(player_minutes, on='gameId', how='inner', rsuffix='_on')
    shots_off_floor = shots_off_floor.join(player_minutes, on='gameId', how='inner', rsuffix='_off')

    # Calculate field goal attempts and percentage
    def calculate_fg_stats(shots, minutes):
        attempts = len(shots)
        made = len(shots[shots['shotResult'] == 'Made'])
        percentage = made / attempts if attempts > 0 else 0
        fg_attempts_per_min = attempts / minutes if minutes > 0 else 0
        fg_made_per_min = made / minutes if minutes > 0 else 0
        return fg_attempts_per_min, fg_made_per_min, percentage

    minutes_on_floor = player_minutes['minutes'].sum()
    fg_attempts_per_min_on, fg_made_per_min_on, fg_percentage_on = calculate_fg_stats(shots_on_floor, minutes_on_floor)

    # Calculate field goal stats when the player is off the floor
    minutes_off_floor = total_minutes_all_games - minutes_on_floor
    fg_attempts_per_min_off, fg_made_per_min_off, fg_percentage_off = calculate_fg_stats(shots_off_floor, minutes_off_floor)

    # print(minutes_on_floor, total_minutes_all_games, minutes_on_floor / total_minutes_all_games)

    # Print the adjusted field goal stats
    # print(f"Player On Floor - FG Attempts per 36 Min: {fg_attempts_per_min_on * 36 * 60:.3f}, FG%: {fg_percentage_on:.2%}")
    # print(f"Player Off Floor - FG Attempts per 36 Min: {fg_attempts_per_min_off * 36 * 60:.3f}, FG%: {fg_percentage_off:.2%}")
    return {
        'player_id': player_id,
        'player_name': player_name,
        'fg_attempts_on': len(shots_on_floor),
        'fg_attempts_per_min_on': fg_attempts_per_min_on * 36 * 60,
        'fg_made_per_min_on': fg_made_per_min_on * 36 * 60,
        'fg_percentage_on': fg_percentage_on,
        'fg_attempts_off': len(shots_off_floor),
        'fg_attempts_per_min_off': fg_attempts_per_min_off * 36 * 60,
        'fg_made_per_min_off': fg_made_per_min_off * 36 * 60,
        'fg_percentage_off': fg_percentage_off,
        'percentage_delta': fg_percentage_on - fg_percentage_off,
    }
players = [
    (1629027, "Young"),
    (1630169, "Haliburton"),
    (203497, "Gobert"),
    (1631117, "Kessler"),
    (1627826, "Zubac"),
    (1630596, "Mobley"),
    (203994, "Nurkic")
]

players_dfpm = []
for player_id, player_name in players:
    result = defpmpaint(player_id, player_name)
    players_dfpm.append(result)

# Step 4: Convert the list of results to a dataframe
players_dfpm = pd.DataFrame(players_dfpm)
players_dfpm.sort_values("percentage_delta").style.format({
    'fg_attempts_per_min_on': '{:,.2f}'.format,
    'fg_made_per_min_on': '{:,.2f}'.format,
    'fg_percentage_on': '{:,.2%}'.format,
    'fg_attempts_per_min_off': '{:,.2f}'.format,
    'fg_made_per_min_off': '{:,.2f}'.format,
    'fg_percentage_off': '{:,.2%}'.format,
    'percentage_delta': '{:,.2%}'.format,
})

### Defensive Rim Field Goals

Now that we have our defensive plus/minus, let's compare it to the existing defensive rim field goals stat and see how the players stack up.

In [ ]:
player_ids = [player[0] for player in players]

# Filter for the players of interest
boxscore_players = boxscore_pt_list[boxscore_pt_list['personId'].isin(player_ids)]

# Group by personId and aggregate the stats
rim_defense = boxscore_players.groupby('personId', observed=True).agg(
    player_name=("familyName", "first"),
    total_defended_at_rim_made=('defendedAtRimFieldGoalsMade', 'sum'),
    total_defended_at_rim_attempted=('defendedAtRimFieldGoalsAttempted', 'sum')
).reset_index()

# Calculate the overall defended field goal percentage at the rim
rim_defense['defended_at_rim_field_goal_percentage'] = (
    rim_defense['total_defended_at_rim_made'] / rim_defense['total_defended_at_rim_attempted'] * 100
)

# Output the result
rim_defense.sort_values("defended_at_rim_field_goal_percentage")

## 4. Data Reporting

To create our final report from our analysis, we will be using Power BI. We have a Microsoft SQL Server database that our Power BI report will import the tables from. If no database is available, the dataframes will instead export as an Excel spreadsheet, which can be manually uploaded to Power BI.

In [ ]:
from pandas import ExcelWriter
import sqlalchemy

dfs = {"defensive_plus_or_minus": players_dfpm, "rim_defense": rim_defense}

def upload_dataframes(dfs: Dict[str, pd.DataFrame]) -> None:
    DATABASE_URL = f"{DB_TYPE}://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}{'?driver=' if DB_DRIVER else ''}{DB_DRIVER.replace(' ', '+') if DB_DRIVER else ''}"

    engine = sqlalchemy.create_engine(DATABASE_URL)

    with engine.connect() as connection:
        print("Connection to the database was successful!")

    inspector = sqlalchemy.inspect(engine)
    existing_tables = inspector.get_table_names()

    for table_name, df in dfs.items():
        # check if table already exists
        if table_name in existing_tables:
            # if there are no changes to the table, do not write to it
            existing_df = pd.read_sql_table(table_name, engine)
            if df.shape == existing_df.shape and df.equals(existing_df):
                print(f"No changes detected for table {table_name}. Skipping upload.")
                continue
        else:
            print(f"Table {table_name} does not exist. Creating a new one.")
        df.to_sql(table_name, engine, if_exists="replace", index=False)
        print(f"Uploaded dataframe to table {table_name}.")

if DB_PASSWORD:
    upload_dataframes(dfs)
else:
    EXCEL_MAX_ROWS = 1048575
    excel_writer = pd.ExcelWriter("../data/dataframes.xlsx")
    for name, df in dfs.items():
        df.head(EXCEL_MAX_ROWS).to_excel(excel_writer, sheet_name=name)
